In [1]:
import numpy as np 
import pandas as pd 

df = pd.read_csv('train_mod_v3.csv')
df


,Unnamed: 0,u,v,t,h,mask2drop
0,0,0,1,27,6,1
1,1,2,3,13,8,1
2,2,4,5,74,2,1
3,3,6,7,36,9,1
4,4,8,9,52,2,1
...,...,...,...,...,...,...
16745086,17414504,2369189,2300645,6,7,1
16745087,17414505,62448,88256,60,0,1
16745088,17414506,919913,1183054,71,0,1
16745089,17414507,1774,881,67,0,1


In [2]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_test.to_csv('test_mod_v3.csv')
del df_test 
#del df


In [3]:
N_TOTAL = max(df['u'].max(), df['v'].max()) + 1
df


,Unnamed: 0,u,v,t,h,mask2drop
0,0,0,1,27,6,1
1,1,2,3,13,8,1
2,2,4,5,74,2,1
3,3,6,7,36,9,1
4,4,8,9,52,2,1
...,...,...,...,...,...,...
16745086,17414504,2369189,2300645,6,7,1
16745087,17414505,62448,88256,60,0,1
16745088,17414506,919913,1183054,71,0,1
16745089,17414507,1774,881,67,0,1


In [4]:
N_TOTAL


2522474

In [5]:
def apk(pred, target, k):
    if len(pred) >= k:
        pred = pred[:k]

    ans, cnt = 0, 0
    tot = min(len(target), k) 
    s = set()
    for i in range(len(pred)):
        if pred[i] in target and pred[i] not in s:
            cnt += 1
            ans += cnt / (i + 1)
            s.add(pred[i])
    return ans / tot

def mapk(pred, target, k):
    assert len(pred) == len(target)
    sum_metric = 0
    for cur_pred, cur_target in zip(pred, target):
        sum_metric += apk(cur_pred, cur_target, k)
    return sum_metric / len(pred) 


In [6]:
df_test = pd.read_csv('test_mod_v3.csv')

target_arr = [[] for _ in range(N_TOTAL)]

for i in range(df_test.shape[0]):
    u, v = df_test['u'].iloc[i], df_test['v'].iloc[i]
    if u >= N_TOTAL or v >= N_TOTAL:
        continue
    target_arr[u].append(v)
    target_arr[v].append(u)
del df_test


In [7]:
import networkx as nx
import random
from networkx import ego_graph
from tqdm.auto import trange


graph = nx.Graph()
graph.add_nodes_from(list(range(N_TOTAL)))
graph.add_edges_from(df_train[['u', 'v']].values)

In [8]:
train_edges_set = set()

for i in trange(df_train.shape[0]):
    u, v = df_train['u'].iloc[i], df_train['v'].iloc[i]
    train_edges_set.add((u, v))
    train_edges_set.add((v, u))

In [9]:
def inference_max(n_first_samples, ranks, K):
    sorted_ranks = sorted([rank, i] for i, rank in enumerate(ranks))[::-1]
    #is2pos = {cur_id: pos for pos, (_, cur_id) in enumerate(sorted_ranks)}
    sum_apk, cnt_good = 0, 0
    for i in trange(n_first_samples):
        if len(target_arr[i]) == 0:
            continue
        recommendations = []
        cur_it = len(sorted_ranks) - 1
        while len(recommendations) < K:
            v0 = sorted_ranks[cur_it][1]
            if (v0, i) not in train_edges_set:
                recommendations.append(v0)
            cur_it -= 1
        sum_apk += apk(recommendations, target_arr[i], K)
        cnt_good += 1
    return sum_apk / cnt_good

In [10]:
pr = nx.pagerank(graph, alpha=0.9)


In [11]:
def inference_nearest(n_first_samples, ranks, K, window_size):
    sorted_ranks = sorted([rank, i] for i, rank in enumerate(ranks))[::-1]
    is2pos = {cur_id: pos for pos, (_, cur_id) in enumerate(sorted_ranks)}
    sum_apk, cnt_good = 0, 0
    for i in trange(n_first_samples):
        if len(target_arr[i]) == 0 or i in list(range(14000, 14100)):
            continue
        recommendations = []
        cur_it = len(sorted_ranks) - 1
        pos_i = is2pos[i]
        for j in range(max(0, pos_i - window_size), min(len(sorted_ranks),  pos_i + window_size + 1)):
            if j != pos_i:
                v0 = sorted_ranks[j][1]
                if (i, v0) in train_edges_set:
                    continue
                try:
                    d0 = nx.shortest_path_length(graph, source=v0, target=i)
                    recommendations.append((d0, v0))
                except:
                    pass
        recommendations = sorted(recommendations)[:K]
        recommendations = [cur_id for _, cur_id in recommendations]

        sum_apk += apk(recommendations, target_arr[i], K)
        cnt_good += 1
    return sum_apk / cnt_good

In [12]:
def inference_nearest_rank(n_first_samples, ranks, K, window_size):
    sorted_ranks = sorted([rank, i] for i, rank in enumerate(ranks))[::-1]
    is2pos = {cur_id: pos for pos, (_, cur_id) in enumerate(sorted_ranks)}
    sum_apk, cnt_good = 0, 0
    for i in trange(n_first_samples):
        if len(target_arr[i]) == 0:
            continue
        recommendations = []
        cur_it = len(sorted_ranks) - 1
        pos_i = is2pos[i]
        for j in range(max(0, pos_i - window_size), min(len(sorted_ranks),  pos_i + window_size + 1)):
            if j != pos_i:
                v0 = sorted_ranks[j][1]
                if (i, v0) in train_edges_set:
                    continue
                recommendations.append((abs(ranks[cur_it] - ranks[j]), v0))

        recommendations = sorted(recommendations)[:K]
        recommendations = [cur_id for _, cur_id in recommendations]
        sum_apk += apk(recommendations, target_arr[i], K)
        cnt_good += 1
    return sum_apk / cnt_good

In [13]:
pr_array = [pr[ind] for ind in range(N_TOTAL)]

In [14]:
inference_max(15000, pr_array, 30)

8.3963237109594e-05

In [15]:
inference_max(N_TOTAL, pr_array, 30)

1.7470726195532689e-06

In [16]:
inference_nearest(15000, pr_array, 30, 30)

0.00033414046568237967

In [17]:
inference_nearest_rank(15000, pr_array, 30, 30)

6.144369510326461e-05

In [18]:
inference_nearest_rank(N_TOTAL, pr_array, 30, 30)

3.0235576830165912e-05